In [0]:
USE CATALOG workspace;
USE SCHEMA sim_retail_demo;

# Bronze Table Queries

In [0]:
CREATE OR REPLACE TABLE customer_bronze_ldp
AS
SELECT *,
  current_timestamp() as processing_time,
  _metadata.file_name
FROM read_files(
  '/Volumes/workspace/sim_retail_demo/customer',
  format => 'json',
  inferSchema => true
);

In [0]:
CREATE OR REPLACE TABLE orders_bronze_ldp
AS
SELECT *,
  current_timestamp() as processing_time,
  _metadata.file_name
FROM read_files(
  '/Volumes/workspace/sim_retail_demo/orders',
  format => 'json',
  inferSchema => true
);

In [0]:
CREATE OR REPLACE TABLE status_bronze_ldp
AS
SELECT *,
  current_timestamp() as processing_time,
  _metadata.file_name
FROM read_files(
  '/Volumes/workspace/sim_retail_demo/status',
  format => 'json',
  inferSchema => true
);

# Silver Table Queries

In [0]:
CREATE OR REPLACE TABLE customer_silver_ldp
AS
SELECT 
  customer_id,
  name,
  email,
  address,
  city,
  state,
  zip_code,
  processing_time as last_updated
FROM customer_bronze_ldp
WHERE operation = 'NEW';

In [0]:
CREATE OR REPLACE TABLE orders_silver_ldp
AS
SELECT 
  order_id,
  customer_id,
  to_timestamp(order_timestamp) as order_timestamp,
  notifications
FROM orders_bronze_ldp;

In [0]:
CREATE OR REPLACE TABLE status_silver_ldp
AS
SELECT 
  order_id,
  order_status,
  to_timestamp(status_timestamp) as status_timestamp
FROM status_bronze_ldp;

# Gold Table Queries

In [0]:
CREATE OR REPLACE VIEW gold_orders_by_city_state_ldp
AS
SELECT
  customers.city,
  customers.state,
  COUNT(orders.order_id) as total_orders
FROM orders_silver_ldp orders
JOIN customer_silver_ldp customers
ON orders.customer_id = customers.customer_id
GROUP BY customers.city, customers.state
ORDER BY total_orders DESC;




In [0]:
CREATE OR REPLACE VIEW gold_returns_by_customer_ldp
AS

WITH returned_orders 
AS (
  SELECT DISTINCT order_id
  FROM status_silver_ldp
  WHERE order_status IN ("return requested", "return picked up", "return processed")
),

order_customer_ids
AS (
  SELECT 
    order_id,
    customer_id
  FROM orders_silver_ldp
)

SELECT 
  customers.customer_id as customer_id,
  customers.city as city,
  customers.state as state,
  COUNT(returned.order_id) as total_returns
FROM returned_orders returned
JOIN order_customer_ids ids
ON returned.order_id = ids.order_id
JOIN customer_silver_ldp customers
ON ids.customer_id = customers.customer_id
GROUP BY customers.customer_id, customers.city, customers.state
ORDER BY total_returns DESC;

In [0]:
CREATE OR REPLACE VIEW gold_customer_order_history_ldp
AS
SELECT 
  customers.customer_id as customer_id,
  orders.order_id as order_id,
  status.order_status as order_status,
  status.status_timestamp as status_update_time
FROM orders_silver_ldp orders
JOIN status_silver_ldp status
ON orders.order_id = status.order_id
JOIN customer_silver_ldp customers
ON orders.customer_id = customers.customer_id;